# THIS IS A JUPYTER NOTEBOOK VERSION OF RT2 ASSIGNMENT

Hi, here is a Jupyter Notebook implementation. 

To start, the necessary libraries must be imported:

In [1]:
import rospy
from std_msgs.msg import String
from geometry_msgs.msg import Twist
from sensor_msgs.msg import LaserScan
from move_base_msgs.msg import MoveBaseAction, MoveBaseGoal, MoveBaseActionGoal, MoveBaseActionFeedback
from actionlib_msgs.msg import GoalID, GoalStatusArray
from nav_msgs.msg import Odometry
from tf.transformations import *

import jupyros as jr
from jupyros import ros3d
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
from IPython.display import display, clear_output

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np
import math
import time
import os

import getch
import sys, tty
sys.path.append("../src/")
import master
import option1
import option2
import option3

Now, lets visualize the robots movement

In [2]:
v = ros3d.Viewer()
ros_connect = ros3d.ROSConnection(url="ws://localhost:9090") 
tf_client = ros3d.TFClient(ros=ros_connect, fixed_frame='map')
laser_view = ros3d.LaserScan(topic="/scan", ros=ros_connect, tf_client=tf_client) 
map_view = ros3d.OccupancyGrid(topic="/map", ros=ros_connect, tf_client=tf_client, continuous=True) 
path = ros3d.Path(topic="/move_base/NavfnROS/plan", ros=ros_connect, tf_client=tf_client) 
urdf = ros3d.URDFModel(ros=ros_connect, tf_client=tf_client, path=os.environ.get('JUPYROS_ASSETS_URL', 'http://localhost:3000')) 
g = ros3d.GridModel()
v.objects = [g, laser_view, map_view, path, urdf]
v

Viewer(objects=[GridModel(), LaserScan(ros=ROSConnection(url='ws://localhost:9090'), tf_client=TFClient(fixed_…

In [3]:
## Push the button

button = master.main()
button

Whats Up? This is RT1 Assignment III, and here is an UI for it.
Press [1] for autonomous drive of the robot
Press [2] for manual drive of the robot
Press [3] for manual drive of the robot with collision avoidance
Press [4] for exitting from the system
Choose drive options: 1
Autonomous drive is chosen
Insert x coordinate: 4
Insert y coordinate: 1
The coordinates were given: x = 4.00 and y = 1.00


IndexError: list index out of range

Plotting the graphs for laser scan, odometry, and goal status

In [4]:
%matplotlib

angles = []

class Graphs:
    def __init__(self):
        
        self.fig_laser = plt.figure(figsize=(6,6))
        self.ax_laser = plt.subplot(111, polar=True)
        self.ax_laser.set_thetalim(-np.pi/2,np.pi/2)
        self.ax_laser.set_theta_zero_location("N")
        self.ln_laser, = self.ax_laser.plot([],[],'bo')
        
        
        self.fig_odom, self.ax_odom = plt.subplots()
        plt.grid(True)
        self.ln_odom, = plt.plot([] ,[], 'bo')
        
        self.achieved = False
        self.goal_counter = 0
        self.non_reached_goal_counter = 0
        self.laser  = [] 
        self.x_pos, self.y_pos = [], []

        self.fig_target, self.ax_target = plt.subplots()
        plt.grid(True)
        self.status_list = [0, 0]
        self.data = {'Reached Goals': self.goal_counter, 
                     'Unreached Goals':self.non_reached_goal_counter, 
                     'Sent Goals': self.goal_counter + self.non_reached_goal_counter}
        
        self.target = list(self.data.keys())
        self.values = list(self.data.values())
        self.ln_target = plt.bar(self.target, self.values, color=("green", "red", "blue"))
        
              
    def plot_init(self):
        self.ax_laser.set_title("Laser of the robot")
        self.ax_laser.set_ylabel("Distance")
        
        self.ax_odom.set_xlim(-20, 20)
        self.ax_odom.set_ylim(-20, 20)

        self.ax_odom.set_title("Position")
        self.ax_odom.set_ylabel("Y-axis")
        self.ax_odom.set_xlabel("X-axis")

        self.ax_target.set_ylim(0, 10)
        
        return self.ln_laser , self.ln_odom, self.ln_target
    
    def laser_scan_callback(self, msg):
        global angles
        angles = list(np.arange(msg.angle_min, msg.angle_max+msg.angle_increment, msg.angle_increment))
        self.laser=list(msg.ranges)
        
    def odometry_callback(self, msg):
        self.y_pos.append(msg.pose.pose.position.y)
        self.x_pos.append(msg.pose.pose.position.x)
        
    def goal_callback(self, msg): 

        try: 

            self.status_list.append(msg.status_list[0].status)

            if self.status_list[-1] == 3 and self.status_list[-2] != 3:
                self.goal_counter = self.goal_counter + 1

            elif self.status_list[-1] == 2 and self.status_list[-2] != 2:
                self.non_reached_goal_counter = self.non_reached_goal_counter + 1

            elif self.status_list[-1] == 4 and self.status_list[-2] != 2:
                self.non_reached_goal_counter = self.non_reached_goal_counter + 1

        except IndexError:
            pass
            
            
    def update_plot_odom(self, frame):
        x = self.x_pos
        y = self.y_pos
        self.ln_odom.set_data(x, y)
        return self.ln_odom
    
    def update_plot_target(self, frame):
        x = self.goal_counter
        y = self.non_reached_goal_counter
        z = self.goal_counter + self.non_reached_goal_counter
        self.updated_data = {'Reached goal': x, 
                             'Non-reached goals': y, 
                             'Total Sent goals': z}
        
        h = list(self.updated_data.values())
        q = list(self.updated_data.keys())
        
        self.ln_target = plt.bar(q, h, color=("green", "red", "blue"))
        
        return self.ln_target
    
    def update_laser(self, frame):
        global angles    
        self.ax_laser.set_rmax(20)
        self.ln_laser.set_data(angles,self.laser)
        return self.ln_laser

Using matplotlib backend: TkAgg


Publishers and subscribers

In [5]:
vis = Graphs()
pub_velocity = rospy.Publisher('/cmd_vel', Twist, queue_size = 1)
sub_scan = rospy.Subscriber("/scan", LaserScan, vis.laser_scan_callback)
sub_odom = rospy.Subscriber("/odom", Odometry, vis.odometry_callback)
sub_status = rospy.Subscriber("/move_base/status", GoalStatusArray, vis.goal_callback)

ani_laser = FuncAnimation(vis.fig_laser, vis.update_laser, blit = True)
ani_odom = FuncAnimation(vis.fig_odom, vis.update_plot_odom, init_func=vis.plot_init)
ani_target = FuncAnimation(vis.fig_target, vis.update_plot_target, init_func=vis.plot_init, interval=100)

plt.show()

Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/matplotlib/cbook/__init__.py", line 216, in process
    func(*args, **kwargs)
  File "/usr/lib/python3/dist-packages/matplotlib/animation.py", line 953, in _start
    self._init_draw()
  File "/usr/lib/python3/dist-packages/matplotlib/animation.py", line 1732, in _init_draw
    self._draw_frame(next(self.new_frame_seq()))
  File "/usr/lib/python3/dist-packages/matplotlib/animation.py", line 1760, in _draw_frame
    self._drawn_artists = sorted(self._drawn_artists,
TypeError: 'Line2D' object is not iterable
Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/matplotlib/cbook/__init__.py", line 216, in process
    func(*args, **kwargs)
  File "/usr/lib/python3/dist-packages/matplotlib/animation.py", line 1269, in _handle_resize
    self._init_draw()
  File "/usr/lib/python3/dist-packages/matplotlib/animation.py", line 1732, in _init_draw
    self._draw_frame(next(self.new_frame_seq()))
  File 